In [22]:
from dl_class import EarlyStopping
import json

# import the data runs_data.json
with open('runs_data.json', 'r') as f:
    runs_data = json.load(f)

In [ ]:
import plotly.graph_objects as go
import numpy as np
from lossmoother import LosSmoother

# Get a loss curve (assume runs_data is loaded)
loss_curve = runs_data[0]['train_loss']

# Initialize the ZLossSmoother
lossmother = LosSmoother()

# Smooth the loss curve
both_losses = [lossmother.update(loss) for loss in loss_curve]#[:200]]

# Compute cumulative minimum of the smoothed loss
cummin_smoothed_loss = [min_loss for _, min_loss in both_losses]
smoothed_loss = [loss for loss, _ in both_losses]

# Prepare x values
x_vals = list(range(len(loss_curve)))

# Plot both the raw and smoothed loss curves and the raw points with Plotly, with button to toggle x-axis to log scale
fig = go.Figure()
# Raw loss line
fig.add_trace(go.Scatter(x=x_vals, y=loss_curve, mode='lines', name='Raw Loss'))
# Raw points as dots
fig.add_trace(go.Scatter(x=x_vals, y=loss_curve, mode='markers', name='Raw Loss Points', marker=dict(size=5, color='rgba(0, 0, 0, 0.3)')))
# Smoothed line
fig.add_trace(go.Scatter(x=x_vals, y=smoothed_loss, mode='lines', name='Smoothed Loss (EMA+Spike)'))
# Cumulative min curve of the smoothed loss, straight solid red line
fig.add_trace(go.Scatter(
    x=x_vals,
    y=cummin_smoothed_loss,
    mode='lines',
    name='Cumulative Min (Smoothed)',
    line=dict(color='red')
))

fig.update_layout(
    title="Loss Curve (Raw + Points vs Smoothed + Cumulative Min)",
    xaxis_title='Step',
    yaxis_title='Loss',
    updatemenus=[
        dict(
            type='buttons',
            showactive=True,
            x=1.15,
            y=1.15,
            buttons=[
                dict(label='Linear X',
                     method='relayout',
                     args=[{'xaxis.type': 'linear'}]),
                dict(label='Log X',
                     method='relayout',
                     args=[{'xaxis.type': 'log'}]),
            ],
            direction='down'
        )
    ]
)
fig.show()


In [12]:
# import plotly.graph_objects as go

# FINAL_STEP = 4300
# ES = EarlyStopping(final_step = FINAL_STEP, warmup = 20, log_step = 0.02)
# id = 10
# loss_list = runs_data[id]['train_loss']
# length = len(loss_list)
# if length >= FINAL_STEP:
#     for loss in loss_list[:FINAL_STEP]:
#         ES.update(loss)
#     fig = go.Figure()
#     fig.add_trace(go.Scatter(x=ES.step, y=ES.loss, mode='lines', name='Loss'))
#     fig.add_trace(go.Scatter(x=ES.step, y=ES.predicted_loss, mode='lines', name='Predicted Loss'))
#     fig.update_layout(xaxis_title='Log10(Step)', yaxis_title='Loss')
#     fig.show()
# else:
#     print('too short')


In [3]:
from dataset import get_dataset
from features import FEATURE_MARKET

features = {
    'last_loss': FEATURE_MARKET['last_loss'],
    'first_derivative_tail_10pct': FEATURE_MARKET['first_derivative_tail_10pct'],
}

dataset = get_dataset(features, path = 'runs_data.json')
dataset = dataset[~dataset['run_id'].isin(sorted(dataset['run_id'].unique())[3:6])]
dataset

registered 1 cases of `last_loss`
registered 3 cases of `first_derivative`
registered 2 cases of `loss_ratio`
registered 1 cases of `aitken_extrapolated_limit`
registered 2 cases of `power_law_exponent`
registered 3 cases of `second_derivative`
registered 2 cases of `tail_power_law_exponent`


,run_id,feature_step_id,target_step_id,feature_step,target_step,target_loss,last_loss,first_derivative_tail_10pct
0,a9fa312754c8461e9e6920f6567958f5,81,122,2.540329,3.360025,0.454336,0.524602,-0.091273
1,a9fa312754c8461e9e6920f6567958f5,81,126,2.540329,3.440122,0.448829,0.524602,-0.091273
2,a9fa312754c8461e9e6920f6567958f5,81,130,2.540329,3.520090,0.447928,0.524602,-0.091273
3,a9fa312754c8461e9e6920f6567958f5,81,134,2.540329,3.600101,0.444542,0.524602,-0.091273
4,a9fa312754c8461e9e6920f6567958f5,85,122,2.620136,3.360025,0.454336,0.519377,-0.067617
...,...,...,...,...,...,...,...,...
945,4f09218fe6d1488985d184547a6e4c4f,125,130,3.420121,3.520090,0.459201,0.462690,-0.076286
946,4f09218fe6d1488985d184547a6e4c4f,125,134,3.420121,3.600101,0.455731,0.462690,-0.076286
947,4f09218fe6d1488985d184547a6e4c4f,129,130,3.500099,3.520090,0.459201,0.459201,-0.066852
948,4f09218fe6d1488985d184547a6e4c4f,129,134,3.500099,3.600101,0.455731,0.459201,-0.066852


In [4]:
# Form the filtered dataset:
# For each unique combination of run_id and feature_step_id, select the row with the highest target_step_id
df = dataset.sort_values('target_step_id', ascending=False)
df = df.groupby(['run_id', 'feature_step_id'], as_index=False).first()
df

,run_id,feature_step_id,target_step_id,feature_step,target_step,target_loss,last_loss,first_derivative_tail_10pct
0,12f488d1c64d4af0a221b06c9baa69b7,81,134,2.540329,3.600101,0.448772,0.521197,-0.066106
1,12f488d1c64d4af0a221b06c9baa69b7,85,134,2.620136,3.600101,0.448772,0.518167,-0.045155
2,12f488d1c64d4af0a221b06c9baa69b7,89,134,2.700704,3.600101,0.448772,0.512020,-0.043480
3,12f488d1c64d4af0a221b06c9baa69b7,93,134,2.780317,3.600101,0.448772,0.499032,-0.128480
4,12f488d1c64d4af0a221b06c9baa69b7,97,134,2.860338,3.600101,0.448772,0.489423,-0.141368
...,...,...,...,...,...,...,...,...
219,faf5d441dde14d659d4a6b3bab7da0cc,117,134,3.260071,3.600101,0.445996,0.457312,-0.037036
220,faf5d441dde14d659d4a6b3bab7da0cc,121,134,3.340047,3.600101,0.445996,0.454451,-0.039137
221,faf5d441dde14d659d4a6b3bab7da0cc,125,134,3.420121,3.600101,0.445996,0.450587,-0.042742
222,faf5d441dde14d659d4a6b3bab7da0cc,129,134,3.500099,3.600101,0.445996,0.448136,-0.039748


In [5]:
df['predicted_loss'] = df.apply(
    lambda row: row['last_loss'] * (row['target_step'] / row['feature_step']) ** (row['first_derivative_tail_10pct'] - 0.275),
    axis=1
)

df

,run_id,feature_step_id,target_step_id,feature_step,target_step,target_loss,last_loss,first_derivative_tail_10pct,predicted_loss
0,12f488d1c64d4af0a221b06c9baa69b7,81,134,2.540329,3.600101,0.448772,0.521197,-0.066106,0.462754
1,12f488d1c64d4af0a221b06c9baa69b7,85,134,2.620136,3.600101,0.448772,0.518167,-0.045155,0.468049
2,12f488d1c64d4af0a221b06c9baa69b7,89,134,2.700704,3.600101,0.448772,0.512020,-0.043480,0.467228
3,12f488d1c64d4af0a221b06c9baa69b7,93,134,2.780317,3.600101,0.448772,0.499032,-0.128480,0.449624
4,12f488d1c64d4af0a221b06c9baa69b7,97,134,2.860338,3.600101,0.448772,0.489423,-0.141368,0.444724
...,...,...,...,...,...,...,...,...,...
219,faf5d441dde14d659d4a6b3bab7da0cc,117,134,3.260071,3.600101,0.445996,0.457312,-0.037036,0.443371
220,faf5d441dde14d659d4a6b3bab7da0cc,121,134,3.340047,3.600101,0.445996,0.454451,-0.039137,0.443872
221,faf5d441dde14d659d4a6b3bab7da0cc,125,134,3.420121,3.600101,0.445996,0.450587,-0.042742,0.443304
222,faf5d441dde14d659d4a6b3bab7da0cc,129,134,3.500099,3.600101,0.445996,0.448136,-0.039748,0.444180


In [43]:
plot_predicted_vs_true_loss(df, n_th = 2)

In [1]:
from dataset_v2 import curve_create_dataset
import numpy as np
df = curve_create_dataset(path='runs_data.json')
df['predicted_loss'] = df.apply(
    lambda row: row['last_loss'] * np.exp(row['delta_steps'] * (0.1153 * row['derivative_3'] - 8.5e-6)),
    axis=1
)

In [2]:
df

,run_id,feature_step,target_step,target_loss,delta_steps,last_loss,derivative_3,curve_id,predicted_loss
0,a9fa312754c8461e9e6920f6567958f5,87,4300,0.447104,4213,0.634545,-1.242636e-03,14,0.334781
1,a9fa312754c8461e9e6920f6567958f5,89,4300,0.447104,4211,0.632575,-1.039067e-03,14,0.368525
2,a9fa312754c8461e9e6920f6567958f5,91,4300,0.447104,4209,0.629028,-1.738801e-03,14,0.261011
3,a9fa312754c8461e9e6920f6567958f5,93,4300,0.447104,4207,0.624600,-2.050334e-03,14,0.222918
4,a9fa312754c8461e9e6920f6567958f5,95,4300,0.447104,4205,0.621378,-1.801918e-03,14,0.250275
...,...,...,...,...,...,...,...,...,...
3225,4f09218fe6d1488985d184547a6e4c4f,3890,4300,0.456692,410,0.457923,-8.905111e-07,6,0.456311
3226,4f09218fe6d1488985d184547a6e4c4f,3981,4300,0.456692,319,0.457133,-3.014178e-06,6,0.455845
3227,4f09218fe6d1488985d184547a6e4c4f,4073,4300,0.456692,227,0.456692,-6.076398e-06,6,0.455739
3228,4f09218fe6d1488985d184547a6e4c4f,4168,4300,0.456692,132,0.456692,-1.305672e-06,6,0.456171


In [7]:
import plotly.graph_objects as go

import json
import numpy as np

# Load runs_data.json
with open('runs_data.json', 'r') as f:
    runs_data = json.load(f)

def get_loss_by_run_id(run_id):
    """Return the 'loss' value for the given run_id from runs_data."""
    for run in runs_data:
        if run.get('run_id') == run_id:
            return [(np.log10(i+1), loss) for i, loss in enumerate(run.get('train_loss'))]
    return None



def plot_predicted_vs_true_loss(df, n_th=0, margin=0.002):
    """
    Plots predicted vs true target loss for the n_th run_id in the dataframe.
    Additionally, plots the "raw_loss" series ('hta'), given as (log10(step), loss) pairs.
    X axis is in linear (step) scale but shown as log axis.
    Also draws two horizontal margin lines at +/- `margin` from the max target loss.

    Plots only "hta" (raw loss) points whose step, in linear scale, falls within [X.min(), X.max()] of the df range.

    Parameters:
        df: DataFrame with columns including run_id, feature_step, target_loss, predicted_loss, target_step.
        n_th: index of run_id to plot.
        margin: margin value to add/subtract from the max target loss for the horizontal lines (default: 0.002).
        hta: list of (step_log10, loss) pairs (can be np.float64), optional. If provided, overlays raw loss trace.
    """
    import numpy as np

    # Pick a run (n_th can be changed to plot other runs)
    selected_run_id = sorted(df['run_id'].unique())[n_th]

    hta = get_loss_by_run_id(selected_run_id)

    run_df = df[df['run_id'] == selected_run_id].sort_values('feature_step')

    # Convert x-axis (feature_step, which is log10) to linear scale
    X_log = run_df['feature_step']
    X = 10 ** X_log  # convert back to linear

    Y_true = run_df['target_loss']
    Y_pred = run_df['predicted_loss']

    # Convert vertical_line from log to linear scale as well
    vertical_line = 10 ** (run_df['target_step'].max())
    horizontal_line = run_df['target_loss'].max()

    # Plot using plotly
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=X, y=Y_true, mode='lines+markers', name='True Target Loss'))
    fig.add_trace(go.Scatter(x=X, y=Y_pred, mode='lines+markers', name='Predicted Loss'))

    # Optionally overlay "hta" (raw loss), only in X range given by df
    if hta is not None:
        hta = list(hta)
        x_hta_full = [10**float(pair[0]) for pair in hta]
        y_hta_full = [float(pair[1]) for pair in hta]
        x_min, x_max = X.min(), X.max()
        # Filter to only those hta points inside the X range
        x_hta, y_hta = zip(*[
            (x, y) for x, y in zip(x_hta_full, y_hta_full)
            if x_min <= x <= x_max
        ]) if len(x_hta_full) > 0 else ([], [])
        if x_hta and y_hta:
            fig.add_trace(go.Scatter(
                x=x_hta,
                y=y_hta,
                mode='lines+markers',
                name='Raw Loss (hta)',
                line=dict(color="rgba(128,128,128,0.7)", width=2, dash="dash")
            ))

    fig.add_shape(
        type="line",
        x0=vertical_line, y0=Y_true.min(),
        x1=vertical_line, y1=horizontal_line,
        line=dict(color="RoyalBlue", width=2, dash="dot"),
        name="Max Target Step"
    )
    fig.add_shape(
        type="line",
        x0=X.min(), y0=horizontal_line,
        x1=vertical_line, y1=horizontal_line,
        line=dict(color="Orange", width=2, dash="dot"),
        name="Max Target Loss"
    )
    # Add margin horizontal lines
    fig.add_shape(
        type="line",
        x0=X.min(), y0=horizontal_line + margin,
        x1=vertical_line, y1=horizontal_line + margin,
        line=dict(color="Green", width=1, dash="dash"),
        name="Max Target Loss + margin"
    )
    fig.add_shape(
        type="line",
        x0=X.min(), y0=horizontal_line - margin,
        x1=vertical_line, y1=horizontal_line - margin,
        line=dict(color="Green", width=1, dash="dash"),
        name="Max Target Loss - margin"
    )
    fig.update_layout(
        title=f'Predicted vs True Target Loss for run_id: {selected_run_id}',
        xaxis_title='Feature Step (linear)',
        yaxis_title='Loss',
        legend=dict(x=0.01, y=0.99),
        xaxis_type="log"  # set x axis to log scale
    )
    fig.show()

In [ ]:
import plotly.graph_objects as go

import json


def plot_predicted_vs_true_loss(df, n_th=0, margin=0.004):
    """
    Plots predicted vs true target loss for the n_th run_id in the dataframe.
    The true target loss is shown as a horizontal line (not scatter points).
    Additionally, plots the "raw_loss" series ("hta"), given as (log10(step), loss) pairs.
    X axis is in linear (step) scale but shown as log axis.
    Also draws two horizontal margin lines at +/- `margin` from the max target loss.

    Plots only "hta" (raw loss) points whose step, in linear scale, falls within [X.min(), X.max()] of the df range.

    Parameters:
        df: DataFrame with columns including run_id, feature_step, target_loss, predicted_loss, target_step.
        n_th: index of run_id to plot.
        margin: margin value to add/subtract from the max target loss for the horizontal lines (default: 0.002).
        hta: list of (step_log10, loss) pairs (can be np.float64), optional. If provided, overlays raw loss trace.
    """
    import numpy as np

    # Pick a run (n_th can be changed to plot other runs)
    selected_run_id = sorted(df['run_id'].unique())[n_th]

    with open('runs_data.json', 'r') as f:
        runs_data = json.load(f)

    # Get train_loss array for this run
    train_loss = None
    for run in runs_data:
        if run.get('run_id') == selected_run_id:
            train_loss = run.get('train_loss')
            break

    run_df = df[df['run_id'] == selected_run_id].sort_values('feature_step')

    # Use feature_step directly as x-axis
    X = run_df['feature_step']
    Y_true = run_df['target_loss']
    Y_pred = run_df['predicted_loss']

    vertical_line = run_df['target_step'].max()
    horizontal_line = run_df['target_loss'].max()

    # Plot using plotly
    fig = go.Figure()

    # Only draw a horizontal line for true target loss (not scatter points)
    fig.add_shape(
        type="line",
        x0=X.min(), y0=horizontal_line,
        x1=vertical_line, y1=horizontal_line,
        line=dict(color="Orange", width=2, dash="dot"),
        name="Max Target Loss"
    )

    # Plot raw loss at feature_step positions
    if train_loss is not None:
        x_raw = []
        y_raw = []
        min_step = int(X.min())
        if min_step - 1 < len(train_loss):
            # Keep every step at and after min_step (feature_step indices start at 1)
            x_raw = list(range(min_step, min_step + len(train_loss[min_step-1:])))
            y_raw = train_loss[min_step-1:4300]
        if x_raw and y_raw:
            fig.add_trace(go.Scatter(
                x=x_raw,
                y=y_raw,
                mode='lines+markers',
                name='Actual Loss',
                line=dict(color="rgba(128,128,128,0.7)", width=2, dash="dash")
            ))

    fig.add_shape(
        type="line",
        x0=vertical_line, y0=Y_true.min(),
        x1=vertical_line, y1=horizontal_line,
        line=dict(color="RoyalBlue", width=2, dash="dot"),
        name="Max Target Step"
    )
    # Add margin horizontal lines
    fig.add_shape(
        type="line",
        x0=X.min(), y0=horizontal_line + margin,
        x1=vertical_line, y1=horizontal_line + margin,
        line=dict(color="Green", width=1, dash="dash"),
        name="Max Target Loss + margin"
    )
    fig.add_shape(
        type="line",
        x0=X.min(), y0=horizontal_line - margin,
        x1=vertical_line, y1=horizontal_line - margin,
        line=dict(color="Green", width=1, dash="dash"),
        name="Max Target Loss - margin"
    )
    fig.update_layout(
        title=f'Predicted vs True Target Loss for run_id: {selected_run_id}',
        xaxis_title='Step',
        yaxis_title='Loss',
        legend=dict(x=0.86, y=0.99),
        xaxis_type="log"  # set x axis to log scale
    )
    # Predicted Loss (blue)
    fig.add_trace(go.Scatter(
        x=X, 
        y=Y_pred, 
        mode='lines+markers', 
        name='Predicted Loss',
        marker=dict(size=9, color="#1f77b4"),             # Larger marker
        line=dict(width=3, color="#1f77b4"),               # Thicker line
    ))
    fig.show()

In [59]:
# Group by run_id and compute the exponential moving average
# of predicted_loss causally (from lowest to highest feature_step)
def ema_causal(df, N=3):
    alpha = 2 / (N + 1)
    df = df.sort_values(['run_id', 'feature_step'])
    df['ema_predicted_loss'] = 0.0
    for run_id, group in df.groupby('run_id', sort=False):
        ema = None
        for idx in group.index:
            x = df.loc[idx, 'predicted_loss']
            if ema is None:
                ema = x
            else:
                ema = alpha * x + (1 - alpha) * ema
            df.loc[idx, 'ema_predicted_loss'] = ema
    return df



# Example usage:
df = ema_causal(df, N=2)
df

,run_id,feature_step,target_step,target_loss,delta_steps,last_loss,derivative_3,curve_id,predicted_loss,ema_predicted_loss
1190,12f488d1c64d4af0a221b06c9baa69b7,87,4300,0.450678,4213,0.573498,-1.246322e-03,0,0.302032,0.302032
1191,12f488d1c64d4af0a221b06c9baa69b7,89,4300,0.450678,4211,0.570773,-1.601646e-03,0,0.253042,0.269372
1192,12f488d1c64d4af0a221b06c9baa69b7,91,4300,0.450678,4209,0.570155,-8.359341e-04,0,0.366667,0.334235
1193,12f488d1c64d4af0a221b06c9baa69b7,93,4300,0.450678,4207,0.570155,-5.572894e-04,0,0.419820,0.391292
1194,12f488d1c64d4af0a221b06c9baa69b7,95,4300,0.450678,4205,0.567215,-9.797697e-04,0,0.340348,0.357329
...,...,...,...,...,...,...,...,...,...,...
845,faf5d441dde14d659d4a6b3bab7da0cc,3890,4300,0.447127,410,0.447838,-1.321581e-07,18,0.446277,0.446161
846,faf5d441dde14d659d4a6b3bab7da0cc,3981,4300,0.447127,319,0.447710,-3.970312e-08,18,0.446497,0.446385
847,faf5d441dde14d659d4a6b3bab7da0cc,4073,4300,0.447127,227,0.447337,-7.835605e-06,18,0.446383,0.446384
848,faf5d441dde14d659d4a6b3bab7da0cc,4168,4300,0.447127,132,0.447256,-1.025741e-05,18,0.446685,0.446584


In [58]:
plot_predicted_vs_true_loss(df, n_th = 12)

In [54]:
plot_predicted_vs_true_loss(df, n_th = 12)

,run_id,feature_step,target_step,target_loss,delta_steps,last_loss,derivative_3,curve_id,predicted_loss,ema_predicted_loss
1190,12f488d1c64d4af0a221b06c9baa69b7,87,4300,0.450678,4213,0.573498,-1.246322e-03,0,0.302032,0.302032
1191,12f488d1c64d4af0a221b06c9baa69b7,89,4300,0.450678,4211,0.570773,-1.601646e-03,0,0.253042,0.277537
1192,12f488d1c64d4af0a221b06c9baa69b7,91,4300,0.450678,4209,0.570155,-8.359341e-04,0,0.366667,0.322102
1193,12f488d1c64d4af0a221b06c9baa69b7,93,4300,0.450678,4207,0.570155,-5.572894e-04,0,0.419820,0.370961
1194,12f488d1c64d4af0a221b06c9baa69b7,95,4300,0.450678,4205,0.567215,-9.797697e-04,0,0.340348,0.355654
...,...,...,...,...,...,...,...,...,...,...
845,faf5d441dde14d659d4a6b3bab7da0cc,3890,4300,0.447127,410,0.447838,-1.321581e-07,18,0.446277,0.446077
846,faf5d441dde14d659d4a6b3bab7da0cc,3981,4300,0.447127,319,0.447710,-3.970312e-08,18,0.446497,0.446287
847,faf5d441dde14d659d4a6b3bab7da0cc,4073,4300,0.447127,227,0.447337,-7.835605e-06,18,0.446383,0.446335
848,faf5d441dde14d659d4a6b3bab7da0cc,4168,4300,0.447127,132,0.447256,-1.025741e-05,18,0.446685,0.446510
